In [1]:
import tensorflow.compat.v2 as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pathlib
import random
import glob
import os
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import time
keras = tf.keras
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image = tf.cast(image, tf.float32)
    image = image/255.0
    image = 2*image-1
    return image

In [3]:
train_dir = r'C:\Users\24306\Desktop\datasets\kfold\1\train'
test_dir  = r'C:\Users\24306\Desktop\datasets\kfold\1\test'
checkpoint_path = r'C:\Users\24306\Desktop\NASNetLarge\1\NASNetLarge.ckpt'

# 处理训练数据

In [4]:
#打乱顺序
train_root = pathlib.Path(train_dir)
train_image_paths = list(train_root.glob('*/*'))
train_image_paths = [str(path) for path in train_image_paths]
random.shuffle(train_image_paths)
#处理训练标签
train_image_labels = [pathlib.Path(path).parent.name for path in train_image_paths]
train_labels = [[label.split('_')[0]] for label in train_image_labels]
new_train_labels=[]
for n in train_labels:
    new_train_labels.append(float(n[0]));  
train_labels=new_train_labels
print(train_labels)
#处理训练图像
train_path_ds = tf.data.Dataset.from_tensor_slices(train_image_paths)
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_image_ds = train_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
train_label_ds = tf.data.Dataset.from_tensor_slices(train_labels)
train_image_label_ds = tf.data.Dataset.zip((train_image_ds, train_label_ds))
# image_count = len(all_image_paths)

# test_count = int(image_count*0.2)
train_count = len(train_image_paths)
# train_data = image_label_ds.skip(test_count)
# test_data = image_label_ds.take(test_count)

BATCH_SIZE = 32
train_data = train_image_label_ds.shuffle(buffer_size=train_count).repeat(-1)
train_data = train_data.batch(BATCH_SIZE)
train_data = train_data.prefetch(buffer_size=AUTOTUNE)

#test_data = test_data.batch(BATCH_SIZE)
print(train_data)



[32.0, 80.0, 23.0, 84.0, 23.0, 37.0, 66.0, 93.0, 52.0, 55.0, 47.0, 47.0, 93.0, 96.0, 37.0, 84.0, 13.0, 7.0, 80.0, 40.0, 37.0, 10.0, 37.0, 4.0, 47.0, 95.0, 37.0, 30.0, 95.0, 66.0, 13.0, 37.0, 52.0, 95.0, 84.0, 95.0, 40.0, 86.0, 35.0, 35.0, 13.0, 4.0, 100.0, 10.0, 13.0, 52.0, 37.0, 30.0, 45.0, 37.0, 66.0, 7.0, 30.0, 16.0, 13.0, 13.0, 52.0, 40.0, 13.0, 35.0, 37.0, 32.0, 52.0, 10.0, 80.0, 95.0, 100.0, 4.0, 100.0, 100.0, 66.0, 4.0, 10.0, 40.0, 100.0, 84.0, 45.0, 7.0, 37.0, 40.0, 95.0, 80.0, 16.0, 13.0, 96.0, 100.0, 16.0, 86.0, 86.0, 37.0, 86.0, 95.0, 13.0, 100.0, 84.0, 40.0, 66.0, 66.0, 86.0, 84.0, 55.0, 37.0, 37.0, 95.0, 45.0, 7.0, 95.0, 84.0, 37.0, 47.0, 16.0, 4.0, 32.0, 37.0, 80.0, 45.0, 52.0, 80.0, 47.0, 7.0, 84.0, 37.0, 32.0, 93.0, 13.0, 10.0, 45.0, 95.0, 23.0, 37.0, 80.0, 95.0, 23.0, 13.0, 16.0, 96.0, 80.0, 100.0, 66.0, 86.0, 84.0, 66.0, 4.0, 86.0, 96.0, 30.0, 66.0, 37.0, 4.0, 95.0, 45.0, 13.0, 96.0, 55.0, 100.0, 86.0, 37.0, 20.0, 80.0, 20.0, 45.0, 16.0, 100.0, 23.0, 45.0, 86.0, 95.0,

# 处理验证数据

In [5]:
#打乱顺序
test_root = pathlib.Path(test_dir)
test_image_paths = list(test_root.glob('*/*'))
test_image_paths = [str(path) for path in test_image_paths]
#random.shuffle(train_image_paths)
#处理训练标签
test_image_labels = [pathlib.Path(path).parent.name for path in test_image_paths]
test_labels = [[label.split('_')[0]] for label in test_image_labels]
new_test_labels=[]
for n in test_labels:
    new_test_labels.append(float(n[0]));  
test_labels=new_test_labels
print(test_labels)
#处理训练图像
test_path_ds = tf.data.Dataset.from_tensor_slices(test_image_paths)
AUTOTUNE = tf.data.experimental.AUTOTUNE
test_image_ds = test_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
test_label_ds = tf.data.Dataset.from_tensor_slices(test_labels)
test_image_label_ds = tf.data.Dataset.zip((test_image_ds, test_label_ds))
# image_count = len(all_image_paths)

# test_count = int(image_count*0.2)
test_count = len(test_image_paths)
# train_data = image_label_ds.skip(test_count)
# test_data = image_label_ds.take(test_count)

BATCH_SIZE = 32
test_data = test_image_label_ds.shuffle(buffer_size=test_count).repeat(-1)
test_data = test_data.batch(BATCH_SIZE)
test_data = test_data.prefetch(buffer_size=AUTOTUNE)

#test_data = test_data.batch(BATCH_SIZE)
print(test_data)

[10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.

In [6]:
#要改
NASNetLarge = tf.keras.applications.nasnet.NASNetLarge(weights='imagenet',include_top=False,input_shape=(224, 224, 3))
#要改
NASNetLarge.trainable = True
inputs = tf.keras.Input(shape=(224, 224, 3))
#要改
x = NASNetLarge(inputs)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
out_weight = tf.keras.layers.Dense(1,name='out_weight')(x)

model = tf.keras.Model(inputs=inputs,
                       outputs=[out_weight])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 NASNet (Functional)         (None, 7, 7, 4032)        84916818  
                                                                 
 global_average_pooling2d (G  (None, 4032)             0         
 lobalAveragePooling2D)                                          
                                                                 
 out_weight (Dense)          (None, 1)                 4033      
                                                                 
Total params: 84,920,851
Trainable params: 84,724,183
Non-trainable params: 196,668
_________________________________________________________________


In [7]:
print ('start time:',time.asctime( time.localtime(time.time()) ))
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, decay=0.01),loss='mse')

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,save_best_only=True,monitor='val_loss')
history = model.fit(train_data,
                    steps_per_epoch=train_count//BATCH_SIZE,
                    epochs=300,
                    validation_data=test_data,
                    validation_steps=test_count//BATCH_SIZE,
                   callbacks=[cp_callback])
#要改
np.savetxt('NASNetLarge_loss_1.txt',history.history['loss'])
#要改
np.savetxt('NASNetLarge_valloss_1.txt',history.history['val_loss'])
print ('end time:',time.asctime( time.localtime(time.time()) ))

start time: Wed Apr 19 21:07:03 2023
Epoch 1/300
591/591 [==============================] - ETA: 0s - loss: 62.6946

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\NASNetLarge\1\NASNetLarge.ckpt\assets


INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\NASNetLarge\1\NASNetLarge.ckpt\assets


591/591 [==============================] - 359s 574ms/step - loss: 62.6946 - val_loss: 4642.5015
Epoch 2/300
591/591 [==============================] - ETA: 0s - loss: 2.5580

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\NASNetLarge\1\NASNetLarge.ckpt\assets


...

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\NASNetLarge\1\NASNetLarge.ckpt\assets


INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\NASNetLarge\1\NASNetLarge.ckpt\assets


591/591 [==============================] - 383s 648ms/step - loss: 0.0057 - val_loss: 0.0662
Epoch 37/300
591/591 [==============================] - 260s 440ms/step - loss: 0.0043 - val_loss: 0.2406
Epoch 38/300
591/591 [==============================] - 261s 442ms/step - loss: 0.0043 - val_loss: 0.3144
Epoch 39/300
591/591 [==============================] - 261s 441ms/step - loss: 0.0044 - val_loss: 0.5225
Epoch 40/300
591/591 [==============================] - 261s 441ms/step - loss: 0.0037 - val_loss: 0.3733
Epoch 41/300
591/591 [==============================] - 262s 444ms/step - loss: 0.0034 - val_loss: 0.5301
Epoch 42/300
591/591 [==============================] - 261s 442ms/step - loss: 0.0038 - val_loss: 0.4941
Epoch 43/300
591/591 [==============================] - 262s 443ms/step - loss: 0.0111 - val_loss: 0.1822
Epoch 44/300
591/591 [==============================] - 264s 447ms/step - loss: 0.0024 - val_loss: 0.2252
Epoch 45/300
591/591 [==============================] - 262

591/591 [==============================] - 270s 457ms/step - loss: 3.7415e-04 - val_loss: 0.2551
Epoch 187/300
591/591 [==============================] - 269s 455ms/step - loss: 3.6036e-04 - val_loss: 0.2908
Epoch 188/300
591/591 [==============================] - 269s 456ms/step - loss: 3.2346e-04 - val_loss: 0.2495
Epoch 189/300
591/591 [==============================] - 270s 457ms/step - loss: 2.7630e-04 - val_loss: 0.2906
Epoch 190/300
591/591 [==============================] - 270s 456ms/step - loss: 3.1606e-04 - val_loss: 0.2631
Epoch 191/300
591/591 [==============================] - 270s 457ms/step - loss: 3.1455e-04 - val_loss: 0.2687
Epoch 192/300
591/591 [==============================] - 270s 456ms/step - loss: 3.3821e-04 - val_loss: 0.2691
Epoch 193/300
591/591 [==============================] - 271s 458ms/step - loss: 5.9124e-04 - val_loss: 0.2654
Epoch 194/300
591/591 [==============================] - 271s 458ms/step - loss: 2.4167e-04 - val_loss: 0.2929
Epoch 195/300
5

591/591 [==============================] - 265s 447ms/step - loss: 1.6944e-04 - val_loss: 0.2744
Epoch 261/300
591/591 [==============================] - 263s 445ms/step - loss: 2.3610e-04 - val_loss: 0.2682
Epoch 262/300
591/591 [==============================] - 265s 448ms/step - loss: 1.9914e-04 - val_loss: 0.2788
Epoch 263/300
591/591 [==============================] - 265s 448ms/step - loss: 1.8424e-04 - val_loss: 0.2823
Epoch 264/300
591/591 [==============================] - 272s 459ms/step - loss: 1.9403e-04 - val_loss: 0.3011
Epoch 265/300
591/591 [==============================] - 267s 452ms/step - loss: 2.0732e-04 - val_loss: 0.2642
Epoch 266/300
591/591 [==============================] - 268s 454ms/step - loss: 1.8549e-04 - val_loss: 0.2836
Epoch 267/300
591/591 [==============================] - 264s 447ms/step - loss: 1.8531e-04 - val_loss: 0.2770
Epoch 268/300
591/591 [==============================] - 273s 462ms/step - loss: 1.6973e-04 - val_loss: 0.2837
Epoch 269/300
5